# 计算交易指标：
## 一、 导入股票price基础数据

In [1]:
import pandas as pd

from util import location

example_data = location.read_from_csv(ts_code="600276.SH", root="../data_store/")
example_data

,ts_code,trade_date,open,high,low,close,pre_close,vol,amount
0,600276.SH,2000-10-18,26.88,28.45,26.50,27.70,11.98,271930.00,748996.054
1,600276.SH,2000-10-19,27.61,28.19,27.01,27.21,27.70,77476.00,214900.952
2,600276.SH,2000-10-20,26.98,27.88,26.30,27.75,27.21,51195.00,138248.375
3,600276.SH,2000-10-23,27.88,28.09,27.40,27.90,27.75,21437.00,59486.626
4,600276.SH,2000-10-24,27.98,28.56,27.61,28.28,27.90,26559.00,74935.642
...,...,...,...,...,...,...,...,...,...
4979,600276.SH,2021-07-07,62.29,63.49,62.26,62.45,62.06,421309.78,2645435.620
4980,600276.SH,2021-07-08,62.46,62.88,61.17,61.37,62.45,376692.99,2325415.956
4981,600276.SH,2021-07-09,61.02,62.32,60.90,62.10,61.37,317139.65,1956917.068
4982,600276.SH,2021-07-12,63.50,64.50,62.99,63.12,62.10,599866.95,3823509.581


In [2]:
example_data.dtypes

ts_code               object
trade_date    datetime64[ns]
open                 float64
high                 float64
low                  float64
close                float64
pre_close            float64
vol                  float64
amount               float64
dtype: object

## 二、计算股票复权价格
1. 参考了很多数据，为了复权价格，我在尽可能的范围内将这个数据免费化
2. 所需要得数据包括但不限于：开、高、收、低
- 昨收（复权的价格：可从交易所直接得到的原始数据）
- 我们所需要的价格是实际交易的价格，而不是复权之后的价格，所以选择自己计算复权价


## 三、计算步骤：

1. 计算涨跌幅</br>
公式：$pc = close \div preclose - 1$

In [3]:
example_data["pct_change"] = example_data["close"] / example_data["pre_close"] - 1
example_data

,ts_code,trade_date,open,high,low,close,pre_close,vol,amount,pct_change
0,600276.SH,2000-10-18,26.88,28.45,26.50,27.70,11.98,271930.00,748996.054,1.312187
1,600276.SH,2000-10-19,27.61,28.19,27.01,27.21,27.70,77476.00,214900.952,-0.017690
2,600276.SH,2000-10-20,26.98,27.88,26.30,27.75,27.21,51195.00,138248.375,0.019846
3,600276.SH,2000-10-23,27.88,28.09,27.40,27.90,27.75,21437.00,59486.626,0.005405
4,600276.SH,2000-10-24,27.98,28.56,27.61,28.28,27.90,26559.00,74935.642,0.013620
...,...,...,...,...,...,...,...,...,...,...
4979,600276.SH,2021-07-07,62.29,63.49,62.26,62.45,62.06,421309.78,2645435.620,0.006284
4980,600276.SH,2021-07-08,62.46,62.88,61.17,61.37,62.45,376692.99,2325415.956,-0.017294
4981,600276.SH,2021-07-09,61.02,62.32,60.90,62.10,61.37,317139.65,1956917.068,0.011895
4982,600276.SH,2021-07-12,63.50,64.50,62.99,63.12,62.10,599866.95,3823509.581,0.016425


2. 计算复权因子：假设我们用单位1去购买了股票，最终会变成多少</br>
公式：$af = \prod{(1 + pct_change)}$

In [4]:
example_data["adj_factor"] = (1 + example_data["pct_change"]).cumprod()
example_data

,ts_code,trade_date,open,high,low,close,pre_close,vol,amount,pct_change,adj_factor
0,600276.SH,2000-10-18,26.88,28.45,26.50,27.70,11.98,271930.00,748996.054,1.312187,2.312187
1,600276.SH,2000-10-19,27.61,28.19,27.01,27.21,27.70,77476.00,214900.952,-0.017690,2.271285
2,600276.SH,2000-10-20,26.98,27.88,26.30,27.75,27.21,51195.00,138248.375,0.019846,2.316361
3,600276.SH,2000-10-23,27.88,28.09,27.40,27.90,27.75,21437.00,59486.626,0.005405,2.328881
4,600276.SH,2000-10-24,27.98,28.56,27.61,28.28,27.90,26559.00,74935.642,0.013620,2.360601
...,...,...,...,...,...,...,...,...,...,...,...
4979,600276.SH,2021-07-07,62.29,63.49,62.26,62.45,62.06,421309.78,2645435.620,0.006284,267.640269
4980,600276.SH,2021-07-08,62.46,62.88,61.17,61.37,62.45,376692.99,2325415.956,-0.017294,263.011743
4981,600276.SH,2021-07-09,61.02,62.32,60.90,62.10,61.37,317139.65,1956917.068,0.011895,266.140284
4982,600276.SH,2021-07-12,63.50,64.50,62.99,63.12,62.10,599866.95,3823509.581,0.016425,270.511670


# 自己实现的类计算复权价


In [5]:
from acquire import stock
from util import location
df = location.read_from_csv(ts_code="600276.SH",root="../data_store/")
ind = stock.Indicators(df)
df = ind.pct_change()
df

,ts_code,trade_date,open,high,low,close,pre_close,vol,amount,pct_change,adj_factor,hfq_close,qfq_close,qfq_open,qfq_high,qfq_low,hfq_open,hfq_high,hfq_low,hfq_close_pct
0,600276.SH,2000-10-18,26.88,28.45,26.50,27.70,11.98,271930.00,748996.054,1.312187,2.312187,27.700000,0.539516,0.523544,0.539516,0.539516,26.880000,27.700000,27.700000,NaN
1,600276.SH,2000-10-19,27.61,28.19,27.01,27.21,27.70,77476.00,214900.952,-0.017690,2.271285,27.210000,0.529972,0.537763,0.529972,0.529972,27.610000,27.210000,27.210000,-0.017690
2,600276.SH,2000-10-20,26.98,27.88,26.30,27.75,27.21,51195.00,138248.375,0.019846,2.316361,27.750000,0.540489,0.525492,0.540489,0.540489,26.980000,27.750000,27.750000,0.019846
3,600276.SH,2000-10-23,27.88,28.09,27.40,27.90,27.75,21437.00,59486.626,0.005405,2.328881,27.900000,0.543411,0.543021,0.543411,0.543411,27.880000,27.900000,27.900000,0.005405
4,600276.SH,2000-10-24,27.98,28.56,27.61,28.28,27.90,26559.00,74935.642,0.013620,2.360601,28.280000,0.550812,0.544969,0.550812,0.550812,27.980000,28.280000,28.280000,0.013620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4979,600276.SH,2021-07-07,62.29,63.49,62.26,62.45,62.06,421309.78,2645435.620,0.006284,267.640269,3206.330428,62.450000,62.290000,62.450000,62.450000,3198.115651,3206.330428,3206.330428,0.006284
4980,600276.SH,2021-07-08,62.46,62.88,61.17,61.37,62.45,376692.99,2325415.956,-0.017294,263.011743,3150.880679,61.370000,62.460000,61.370000,61.370000,3206.843852,3150.880679,3150.880679,-0.017294
4981,600276.SH,2021-07-09,61.02,62.32,60.90,62.10,61.37,317139.65,1956917.068,0.011895,266.140284,3188.360602,62.100000,61.020000,62.100000,62.100000,3132.910853,3188.360602,3188.360602,0.011895
4982,600276.SH,2021-07-12,63.50,64.50,62.99,63.12,62.10,599866.95,3823509.581,0.016425,270.511670,3240.729810,63.120000,63.500000,63.120000,63.120000,3260.239907,3240.729810,3240.729810,0.016425
